In [ ]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf

In [ ]:
INPUT_PATH = "../data/base.parquet"
MODEL_PATH = "https://tfhub.dev/google/universal-sentence-encoder/4"
OUTPUT_PATH = "../data/embeddings.parquet"

In [ ]:
def tf_hub_embedder(df, col, model, batch_size=512):
    dataset = tf.data.Dataset.from_tensor_slices(df[col].values)
    dataset = dataset.batch(batch_size)
    embeddings = []
    total = len(df)
    current = 0
    for batch in dataset:
        embedding = model(batch)
        embeddings.append(embedding.numpy())
        current += batch_size
        print("{}/{}".format(current, total), end="\r")
    return np.vstack(embeddings)

In [ ]:
df = pd.read_parquet("../data/base.parquet")

In [ ]:
embed = hub.load(MODEL_PATH)

In [ ]:
embeddings = tf_hub_embedder(df, "body", embed)

In [ ]:
pd.DataFrame(zip(df.uuid, embeddings), columns=["uuid","body_embedding"]).to_parquet(OUTPUT_PATH)